In [1]:
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
                         summarize,
                         poly)
from sklearn.model_selection import train_test_split

In [2]:
from functools import partial
from sklearn.model_selection import (
    cross_validate,
    KFold,
    ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm

In [4]:
Portfolio = load_data('Portfolio')
def alpha_func(D, idx):
    cov_ = np.cov(D[['X','Y']].loc[idx], rowvar=False)
    return ((cov_[1,1] - cov_[0 ,1]) /
            (cov_[0 ,0] + cov_[1 ,1]-2* cov_[0 ,1]))

In [5]:
alpha_func(Portfolio , range(100))

0.57583207459283

In [6]:
rng = np.random.default_rng (0)
alpha_func(Portfolio,
           rng.choice(100,
                      100,
                      replace=True))

0.6074452469619004

In [7]:
def boot_SE(func,
            D,
            n=None,
            B=1000,
            seed =0):
    rng = np.random.default_rng(seed)
    first_ , second_ = 0, 0
    n = n or D.shape [0]
    for _ in range(B):
        idx = rng.choice(D.index,
                         n,
                         replace=True)
        value = func(D, idx)
        first_ += value
        second_ += value **2
    return np.sqrt(second_ / B - (first_ / B)**2)

In [8]:
alpha_SE = boot_SE(alpha_func,
                   Portfolio,
                   B=1000,
                   seed=0)
alpha_SE

0.09118176521277699

In [9]:
def boot_OLS(model_matrix, response, D, idx):
    D_ = D.loc[idx]
    Y_ = D_[response]
    X_ = clone(model_matrix).fit_transform(D_)
    return sm.OLS(Y_ , X_).fit().params

In [10]:
hp_func = partial(boot_OLS , MS(['horsepower ']), 'mpg')

In [12]:
Auto = load_data('Auto')
Auto_train , Auto_valid = train_test_split (Auto,
                                            test_size=196,
                                            random_state=0)

In [ ]:
rng = np.random.default_rng(0)
np.array([hp_func(Auto,
                  rng.choice(392,
                             392,
                             replace=True)) for _ in range (10)])

In [ ]:
hp_se = boot_SE(hp_func,
                Auto,
                B=1000,
                seed=10)
hp_se

---